HEURISTIC SEARCH


In [1]:
import heapq
import numpy as np

class Puzzle:
    def __init__(self, initial, target):
        self.initial = tuple(map(tuple, initial))
        self.target = tuple(map(tuple, target))

    def heuristic(self, state):
        """Calculate the number of misplaced tiles as heuristic"""
        return sum(state[i][j] != self.target[i][j] and state[i][j] != 0 for i in range(3) for j in range(3))

    def get_neighbors(self, state):
        """Generate possible moves by swapping the empty tile (0)"""
        neighbors = []
        state = [list(row) for row in state]
        x, y = [(i, row.index(0)) for i, row in enumerate(state) if 0 in row][0]
        moves = {"Up": (-1, 0), "Down": (1, 0), "Left": (0, -1), "Right": (0, 1)}

        for move, (dx, dy) in moves.items():
            nx, ny = x + dx, y + dy
            if 0 <= nx < 3 and 0 <= ny < 3:
                new_state = [row[:] for row in state]
                new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
                neighbors.append((move, tuple(map(tuple, new_state))))
        return neighbors

    def solve(self):
        """Solve the puzzle using A* search"""
        pq = [(self.heuristic(self.initial), 0, self.initial, [])]
        visited = set()

        while pq:
            # Extract the best node
            _, g, state, path = heapq.heappop(pq)
            print(f"\nCurrently exploring state with g(n)={g}, h(n)={self.heuristic(state)}, f(n)={g+self.heuristic(state)}:")
            print(np.array(state))

            if state == self.target:
                return path + [state]

            if state in visited:
                continue
            visited.add(state)

            neighbors = self.get_neighbors(state)
            neighbor_info = []

            print("\nGenerated Neighbors:")
            for move, neighbor in neighbors:
                h_val = self.heuristic(neighbor)
                f_val = g + 1 + h_val
                neighbor_info.append((f_val, g + 1, neighbor, path + [state]))
                print(f"Move: {move}")
                print(np.array(neighbor))
                print(f"h(n) = {h_val}, g(n) = {g+1}, f(n) = {f_val}\n")

            # Find the neighbor with the lowest f(n)
            if neighbor_info:
                best = min(neighbor_info, key=lambda x: x[0])
                print(f"--> Choosing move with lowest f(n) = {best[0]}")

            for info in neighbor_info:
                heapq.heappush(pq, info)

        return None

# Initial and Target Matrix
initial_matrix = [[2, 8, 3],
                  [1, 6, 4],
                  [7, 0, 5]]

target_matrix = [[1, 2, 3],
                 [8, 0, 4],
                 [7, 6, 5]]

puzzle = Puzzle(initial_matrix, target_matrix)

print("Initial State:\n", np.array(initial_matrix))
solution = puzzle.solve()

if solution:
    print("\n\nFinal Solution Path:")
    for step in solution:
        print(np.array(step))
        print()
else:
    print("No solution found")


Initial State:
 [[2 8 3]
 [1 6 4]
 [7 0 5]]

Currently exploring state with g(n)=0, h(n)=4, f(n)=4:
[[2 8 3]
 [1 6 4]
 [7 0 5]]

Generated Neighbors:
Move: Up
[[2 8 3]
 [1 0 4]
 [7 6 5]]
h(n) = 3, g(n) = 1, f(n) = 4

Move: Left
[[2 8 3]
 [1 6 4]
 [0 7 5]]
h(n) = 5, g(n) = 1, f(n) = 6

Move: Right
[[2 8 3]
 [1 6 4]
 [7 5 0]]
h(n) = 5, g(n) = 1, f(n) = 6

--> Choosing move with lowest f(n) = 4

Currently exploring state with g(n)=1, h(n)=3, f(n)=4:
[[2 8 3]
 [1 0 4]
 [7 6 5]]

Generated Neighbors:
Move: Up
[[2 0 3]
 [1 8 4]
 [7 6 5]]
h(n) = 3, g(n) = 2, f(n) = 5

Move: Down
[[2 8 3]
 [1 6 4]
 [7 0 5]]
h(n) = 4, g(n) = 2, f(n) = 6

Move: Left
[[2 8 3]
 [0 1 4]
 [7 6 5]]
h(n) = 3, g(n) = 2, f(n) = 5

Move: Right
[[2 8 3]
 [1 4 0]
 [7 6 5]]
h(n) = 4, g(n) = 2, f(n) = 6

--> Choosing move with lowest f(n) = 5

Currently exploring state with g(n)=2, h(n)=3, f(n)=5:
[[2 0 3]
 [1 8 4]
 [7 6 5]]

Generated Neighbors:
Move: Down
[[2 8 3]
 [1 0 4]
 [7 6 5]]
h(n) = 3, g(n) = 3, f(n) = 6

Move: Left

A* SEARCH


In [3]:
import heapq

class Graph:
    def __init__(self):
        self.graph = {}
        self.heuristics = {}

    def add_edge(self, u, v):
        if u not in self.graph:
            self.graph[u] = []
        if v not in self.graph:
            self.graph[v] = []
        self.graph[u].append(v)
        self.graph[v].append(u)

    def set_heuristic(self, h_values):
        self.heuristics = h_values

    def a_star_search(self, start, goal):
        open_set = [(self.heuristics[start], 0, start, [])]  # (f, g, node, path)
        closed_set = set()
        level = 0

        while open_set:
            print(f"\n--- Level {level} ---")
            print("Open set nodes with (g(n), h(n), f(n)):")

            for f_val, g_val, node, path in open_set:
                print(f"Node {node}: g={g_val}, h={self.heuristics[node]}, f={f_val}")

            f, g, node, path = heapq.heappop(open_set)
            print(f"\n>>> Expanding node {node} with f(n) = {f}")

            if node == goal:
                return path + [node]

            if node in closed_set:
                continue
            closed_set.add(node)

            neighbors = self.graph.get(node, [])
            for neighbor in neighbors:
                if neighbor not in closed_set:
                    g_new = g + 1  # assuming cost between nodes = 1
                    f_new = g_new + self.heuristics[neighbor]
                    heapq.heappush(open_set, (f_new, g_new, neighbor, path + [node]))

            level += 1

        return None

# Initialize the graph
graph = Graph()

# Define edges
graph.add_edge("S", "A")
graph.add_edge("S", "B")
graph.add_edge("A", "C")
graph.add_edge("A", "D")
graph.add_edge("B", "E")
graph.add_edge("B", "F")
graph.add_edge("E", "H")
graph.add_edge("F", "I")
graph.add_edge("F", "G")  # Goal Node

# Set heuristic values
heuristic_values = {"S": 14, "A": 12, "B": 5, "C": 7, "D": 3, "E": 8, "F": 2, "H": 4, "I": 9, "G": 0}
graph.set_heuristic(heuristic_values)

# Run A* search
start_node = "S"
goal_node = "G"
print(f"Finding the shortest path from {start_node} to {goal_node} using A* Search:\n")
solution_path = graph.a_star_search(start_node, goal_node)

if solution_path:
    print("\nSolution path:", " → ".join(solution_path))
else:
    print("No path found")


Finding the shortest path from S to G using A* Search:


--- Level 0 ---
Open set nodes with (g(n), h(n), f(n)):
Node S: g=0, h=14, f=14

>>> Expanding node S with f(n) = 14

--- Level 1 ---
Open set nodes with (g(n), h(n), f(n)):
Node B: g=1, h=5, f=6
Node A: g=1, h=12, f=13

>>> Expanding node B with f(n) = 6

--- Level 2 ---
Open set nodes with (g(n), h(n), f(n)):
Node F: g=2, h=2, f=4
Node A: g=1, h=12, f=13
Node E: g=2, h=8, f=10

>>> Expanding node F with f(n) = 4

--- Level 3 ---
Open set nodes with (g(n), h(n), f(n)):
Node G: g=3, h=0, f=3
Node E: g=2, h=8, f=10
Node I: g=3, h=9, f=12
Node A: g=1, h=12, f=13

>>> Expanding node G with f(n) = 3

Solution path: S → B → F → G


BEST FIRST SEARCH

In [4]:
import heapq

class Graph:
    def __init__(self):
        self.graph = {}
        self.heuristics = {}

    def add_edge(self, u, v):
        if u not in self.graph:
            self.graph[u] = []
        if v not in self.graph:
            self.graph[v] = []
        self.graph[u].append(v)
        self.graph[v].append(u)

    def set_heuristic(self, h_values):
        self.heuristics = h_values

    def best_first_search(self, start, goal):
        open_list = [(self.heuristics[start], start, [])]
        closed_list = set()
        level = 0

        while open_list:
            print(f"\nLevel {level}")
            print("Open list nodes with h(n):")
            for h_val, node, path in open_list:
                print(f"Node {node}: h = {h_val}")

            h, node, path = heapq.heappop(open_list)
            print(f"\nExpanding node {node} with h(n) = {h}")

            if node == goal:
                return path + [node]

            if node in closed_list:
                continue
            closed_list.add(node)

            neighbors = self.graph.get(node, [])
            for neighbor in neighbors:
                if neighbor not in closed_list:
                    heapq.heappush(open_list, (self.heuristics[neighbor], neighbor, path + [node]))

            level += 1

        return None

graph = Graph()

graph.add_edge("S", "A")
graph.add_edge("S", "B")
graph.add_edge("A", "C")
graph.add_edge("A", "D")
graph.add_edge("B", "E")
graph.add_edge("B", "F")
graph.add_edge("E", "H")
graph.add_edge("F", "I")
graph.add_edge("F", "G")

heuristic_values = {
    "S": 14, "A": 12, "B": 5, "C": 7, "D": 3,
    "E": 8, "F": 2, "H": 4, "I": 9, "G": 0
}
graph.set_heuristic(heuristic_values)

start_node = "S"
goal_node = "G"
print(f"\nFinding the path from {start_node} to {goal_node} using Best-First Search:\n")
solution_path = graph.best_first_search(start_node, goal_node)

if solution_path:
    print("\nFinal Path:", " → ".join(solution_path))
else:
    print("\nNo path found")



Finding the path from S to G using Best-First Search:


Level 0
Open list nodes with h(n):
Node S: h = 14

Expanding node S with h(n) = 14

Level 1
Open list nodes with h(n):
Node B: h = 5
Node A: h = 12

Expanding node B with h(n) = 5

Level 2
Open list nodes with h(n):
Node F: h = 2
Node A: h = 12
Node E: h = 8

Expanding node F with h(n) = 2

Level 3
Open list nodes with h(n):
Node G: h = 0
Node E: h = 8
Node I: h = 9
Node A: h = 12

Expanding node G with h(n) = 0

Final Path: S → B → F → G
